# Register Model

## Notebook Overview

- Start Execution
- Install and Import Libraries
- Configure Settings
- Log the Model to MLFlow
- Fetch the Latest Model Version from MLflow
- Load the Model and Run Inference

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-07-26 06:17:04 - INFO - Notebook execution started.


# Install and Import Libraries

In [3]:
%%time

%pip install -r ../requirements.txt --quiet 

Note: you may need to restart the kernel to use updated packages.
CPU times: user 49.1 ms, sys: 9.81 ms, total: 58.9 ms
Wall time: 1.43 s


In [4]:
import datetime
import os
import uuid
import base64
import sys
from typing import Dict, Any, List
import pandas as pd
import warnings
import mlflow
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define the relative path to the 'src' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from core.chatbot_service.chatbot_service import ChatbotService
from src.utils import (
    get_context_window, 
    dynamic_retriever, 
    format_docs_with_adaptive_context,
    login_huggingface,
    load_config,
    load_secrets,
    clean_code
)
from src.prompt_templates import format_rag_chatbot_prompt

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


# Configure Settings

In [5]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

In [6]:
# Configuration parameters for the RAG chatbot
MODEL_SOURCE = "local"  # Can be: "local", "hugging-face-local", "hugging-face-cloud"
CHATBOT_SERVICE_NAME = "VanillaRAGChatbot"

# Configuration paths
CONFIG_PATH = "../configs/config.yaml"
DATA_PATH = "../data"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Chatbot-Run"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "AIStudio-Chatbot-Model"

In [7]:
# Load configuration and secrets
config = load_config(CONFIG_PATH)
secrets = load_secrets()

print("✅ Configuration loaded successfully")
print("✅ Secrets loaded successfully")

✅ Configuration loaded successfully
✅ Secrets loaded successfully


# Register and Log the Model to MLFlow 

In this section, we demonstrate how to deploy a RAG-based chatbot service with integrated Galileo Protect and Observe capabilities. This service provides a REST API endpoint that allows users to query the knowledge base with natural language questions, upload new documents to the knowledge base, and manage conversation history, all with built-in safeguards against sensitive information and toxicity. This service encapsulates all the functionality we developed in this notebook, including the document retrieval system and the RAG-based question answering capabilities. It demonstrates how to use our ChatbotService from the src/service directory. 

In [8]:
mlflow.set_tracking_uri('/phoenix/mlflow')
# === Set MLflow experiment context ===
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# === Validate local model file path ===
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

# === Log and register model to MLflow ===
with mlflow.start_run(run_name=MLFLOW_RUN_NAME) as run:
    
    # Log model artifacts using custom ChatbotService
    ChatbotService.log_model(
        artifact_path=MLFLOW_MODEL_NAME,
        config_path=CONFIG_PATH,
        docs_path=DATA_PATH,
        model_path=LOCAL_MODEL_PATH,
        demo_folder=DEMO_FOLDER
    )

    # Construct the URI for the logged model
    model_uri = f"runs:/{run.info.run_id}/{MLFLOW_MODEL_NAME}"

    # Register the model into MLflow Model Registry
    mlflow.register_model(
        model_uri=model_uri,
        name=MLFLOW_MODEL_NAME
    )

    logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

2025-07-26 06:17:17,443 - INFO - PyTorch version 2.7.1 available.
2025-07-26 06:17:17,896 - INFO - Use pytorch device_name: cuda
2025-07-26 06:17:17,898 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


2025-07-26 06:20:59,079 - INFO - Model and artifacts successfully registered in MLflow.
Successfully registered model 'AIStudio-Chatbot-Model'.
Created version '1' of model 'AIStudio-Chatbot-Model'.
2025-07-26 06:20:59 - INFO - ✅ Model registered successfully with run ID: 5d13db299c2640d79c050a028e647c81


In [9]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-26 06:20:59 - INFO - ⏱️ Total execution time: 3m 54.51s
2025-07-26 06:20:59 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).